# 🌊 Marine Carbonate System - TA & DIC

**Interactive Bjerrum Plot** using Total Alkalinity and DIC parameters.

**Quick Instructions:**
- Adjust sliders to change TA and DIC values
- Observe real-time carbonate system changes
- Use preset buttons for common scenarios

---
**Author:** Cardoso-Mohedano J.G.  
**Institution:** Instituto de Ciencias del Mar y Limnología, UNAM

In [13]:
# Essential imports only
import PyCO2SYS as pyco2
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import FloatSlider, VBox, interactive_output, Button, HTML
from IPython.display import display

In [14]:
# Simplified calculation function
def calc_system(alk, dic):
    """Calculate carbonate system from TA and DIC"""
    results = pyco2.sys(par1=alk, par2=dic, par1_type=1, par2_type=2,
                       salinity=35, temperature=25, pressure=0,
                       opt_pH_scale=1, opt_k_carbonic=10)
    return {
        'pH': float(results['pH_total']),
        'pCO2': float(results['pCO2']),
        'HCO3': float(results['bicarbonate']),
        'CO3': float(results['carbonate']),
        'DIC': float(results['dic']),
        'omega': float(results['saturation_aragonite'])
    }

def plot_system(data):
    """Create simplified 4-panel plot with marine color palette"""
    # Paleta "Colores del Mar"
    azul_profundo = '#023859'    # Fondo o trazos dominantes
    verde_aqua = '#2A9D8F'       # Resaltado o barras positivas
    azul_claro = '#87C4D9'       # Rellenos suaves o líneas auxiliares
    arena_calida = '#E9C46A'     # Resaltado neutro o categorías
    coral_vivo = '#F4A261'       # Alertas o valores negativos
    rojo_atardecer = '#E76F51'   # Advertencias o zonas críticas
    
    plt.close('all')
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 8))
    fig.patch.set_facecolor('white')
    
    # Bjerrum plot
    pH_range = np.linspace(5, 10, 30)
    H = 10**(-pH_range)
    K1, K2 = 1e-6, 1e-9  # Simplified constants
    denom = H**2 + K1*H + K1*K2
    
    ax1.plot(pH_range, H**2/denom, color=azul_profundo, linewidth=2.5, label='CO2')
    ax1.plot(pH_range, K1*H/denom, color=verde_aqua, linewidth=2.5, label='HCO3-')
    ax1.plot(pH_range, K1*K2/denom, color=azul_claro, linewidth=2.5, label='CO3-2')
    ax1.axvline(data['pH'], color=coral_vivo, linestyle='--', linewidth=2)
    ax1.set_xlabel('pH'), ax1.set_ylabel('Fraction')
    ax1.set_title('Bjerrum Plot'), ax1.legend(), ax1.grid(True, alpha=0.3)
    
    # Composition pie
    CO2_conc = data['pCO2'] * 0.034
    species = [CO2_conc, data['HCO3'], data['CO3']]
    colors_pie = [azul_profundo, verde_aqua, azul_claro]
    ax2.pie(species, labels=['CO2*', 'HCO3-', 'CO3-2'], autopct='%1.1f%%', colors=colors_pie)
    ax2.set_title(f'Composition at pH {data["pH"]:.2f}')
    
    # Saturation
    omega = data['omega']
    color = verde_aqua if omega >= 1 else rojo_atardecer
    ax3.bar(['Aragonite'], [omega], color=color)
    ax3.axhline(1, color=azul_profundo, linestyle='--')
    ax3.set_ylabel('Omega'), ax3.set_title('Saturation State')
    
    # Results
    ax4.axis('off')
    results_text = f"""pH = {data['pH']:.2f}
pCO2 = {data['pCO2']:.0f} μatm
HCO3- = {data['HCO3']:.0f} μmol/kg
CO3-2 = {data['CO3']:.0f} μmol/kg
DIC = {data['DIC']:.0f} μmol/kg
Ω_arag = {omega:.2f}

Conditions:
S = 35 PSU, T = 25°C"""
    ax4.text(0.1, 0.9, results_text, transform=ax4.transAxes, 
             fontsize=10, verticalalignment='top', fontfamily='monospace', color=azul_profundo)
    
    plt.tight_layout()
    plt.show()
    return fig

In [ ]:
# Create interface
alk_slider = FloatSlider(value=2300, min=1800, max=3000, step=50,
                        description="Alkalinity:", style={'description_width': 'initial'})
dic_slider = FloatSlider(value=2020, min=1600, max=2800, step=50,
                        description="DIC:", style={'description_width': 'initial'})

def update_plot(alk, dic):
    """Update plot when sliders change"""
    data = calc_system(alk, dic)
    plot_system(data)

# Preset buttons
def reset_vals(b): alk_slider.value, dic_slider.value = 2300, 2020
def ocean_vals(b): alk_slider.value, dic_slider.value = 2300, 2020
def surface_vals(b): alk_slider.value, dic_slider.value = 2300, 1900
def deep_vals(b): alk_slider.value, dic_slider.value = 2350, 2200

reset_btn = Button(description="Reset", button_style='info')
ocean_btn = Button(description="Ocean", button_style='success')
surface_btn = Button(description="Surface", button_style='warning')
deep_btn = Button(description="Deep", button_style='danger')

reset_btn.on_click(reset_vals)
ocean_btn.on_click(ocean_vals)
surface_btn.on_click(surface_vals)
deep_btn.on_click(deep_vals)

# Display interface
display(VBox([
    HTML("<h3>🌊 Marine Carbonate System Explorer</h3>"),
    alk_slider, dic_slider,
    VBox([ocean_btn, surface_btn, deep_btn, reset_btn]),
    interactive_output(update_plot, {'alk': alk_slider, 'dic': dic_slider})
]))

print("✅ Fast carbonate system interface ready!")
print("🎯 Optimized for quick Binder loading")

✅ Fast carbonate system interface ready!
🎯 Optimized for quick Binder loading
